# Alive & Thrive
## Viet Nam Data: Children Files - Tabulation
## Prepared by Aaron Wise; aaron@a3di.dev
### Version: 14 July 2022

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_columns", None)

import statsmodels.api as sm

In [2]:
# %load tabulate_imports.py
from tabulate_vars import (
    read_csv_file,
    concatenate_dfs,
    save_combined,
    create_bivariate_var_dep,
    extract_regression_params
)

from aw_analytics import mean_wt, output_mean_table

### Create and save combined children's file

In [3]:
# Set parameters
country = 'LAO'
recode = 'children'

# -------------------------------------------------------------------
year_list = ['2000', '2006', '2012', '2017'] 
# -------------------------------------------------------------------


# Read in files, store dfs in list
list_of_dfs = [read_csv_file(country, recode, year, file_type='working') for year in year_list]

# Combine the dfs
df = concatenate_dfs(list_of_dfs)

# Save and export
save_combined(df, country, recode)

The file -- LAO_children_2000_working.csv -- has the following shape: Rows: 5152; Columns: 31
The file -- LAO_children_2006_working.csv -- has the following shape: Rows: 4136; Columns: 25
The file -- LAO_children_2012_working.csv -- has the following shape: Rows: 11067; Columns: 34
The file -- LAO_children_2017_working.csv -- has the following shape: Rows: 11720; Columns: 46


### Run Descriptive stats

In [4]:
## TOTAL DATASET

# Set parameters
vars = ['excl_bf', 'cont_1223_bf', 'mdd_ch', 'mmf_ch', 'mad_ch', 'stunting_ch', 'wasting_ch', 'overweight_ch'] 

ind_vars = ['Total', 'sex_ch', 'residence', 'region', 'mother_edu', 'wealth_q', 'eth_hoh', 'elderly_hoh', 'sex_hoh']

wt = 'chweight'

# Run for loop

for year in year_list:

    df = read_csv_file(country, recode, year, file_type='working')

    out_fn = country + "_" + recode + "_" + year + ".xlsx"
    path = Path.cwd() / 'output' / 'frequencies' / recode / out_fn
    
    # Instantiate object
    xlwriter = pd.ExcelWriter(path)
    
    for var in vars:

        try:
            # df = df.dropna(subset=[var])
            output = output_mean_table(df.dropna(subset=[var]), var, ind_vars, wt)
            sheet_name = str(var) + '_weighted'
            output.to_excel(xlwriter, sheet_name=sheet_name)
        
        except:
            print(f"Warning: Unable to create sheet for var {var}")

    xlwriter.close()

The file -- LAO_children_2000_working.csv -- has the following shape: Rows: 5152; Columns: 31
The file -- LAO_children_2006_working.csv -- has the following shape: Rows: 4136; Columns: 25
The file -- LAO_children_2012_working.csv -- has the following shape: Rows: 11067; Columns: 34
The file -- LAO_children_2017_working.csv -- has the following shape: Rows: 11720; Columns: 46


### Run Bivariate stats

In [5]:
# Read in combined file
df = read_csv_file(country, recode, file_type='combined')

# Create updated bivariate variables (mother_edu_biv, eth_hoh_biv)
df = create_bivariate_var_dep(df, country)

The file -- LAO_children_combined.csv -- has the following shape: Rows: 32075; Columns: 52


c:\Users\Lindsey\OneDrive - A3DI\Alive & Thrive\3_Country_II\tabulate_vars.py:38: DtypeWarning: Columns (24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [6]:
# Set parameters
recode = 'children'

var_dep_list = ['excl_bf', 'cont_1223_bf', 'mdd_ch', 'mmf_ch', 'mad_ch', 'stunting_ch', 'wasting_ch', 'overweight_ch']

ind_var_list = ['sex_ch', 'residence', 'region', 'mother_edu_biv', 'wealth_q', 'eth_hoh_biv', 'elderly_hoh', 'sex_hoh']

# Run bivariate (extract WLS regression params)
output = {var_dep: pd.DataFrame({ind_var: extract_regression_params(df, var_dep, ind_var, recode) for ind_var in ind_var_list}) for var_dep in var_dep_list}

sex_ch
                            WLS Regression Results                            
Dep. Variable:                excl_bf   R-squared:                       0.047
Model:                            WLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     28.30
Date:                Thu, 17 Nov 2022   Prob (F-statistic):           7.38e-18
Time:                        12:12:26   Log-Likelihood:                -9130.6
No. Observations:                1714   AIC:                         1.827e+04
Df Residuals:                    1710   BIC:                         1.829e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
intercept              

In [7]:
# Generate csv of outputs
for var in var_dep_list:
    fn = var + '.csv'
    output[var].transpose().to_csv(f"./output/bivariate/{recode}/{fn}")

#### Get bivariate differences

In [8]:
df = read_csv_file(country, recode, file_type='combined')

df = create_bivariate_var_dep(df, country)

var_dep_list = ['excl_bf', 'cont_1223_bf', 'mdd_ch', 'mmf_ch', 'mad_ch', 'stunting_ch', 'wasting_ch', 'overweight_ch'] 

ind_var_list = ['sex_ch', 'residence', 'region', 'mother_edu_biv', 'wealth_q', 'eth_hoh_biv', 'elderly_hoh', 'sex_hoh']


for var in var_dep_list:

    out_fn = var + "_bivariate_diff" + ".xlsx"
    path = Path.cwd() / 'output' / 'bivariate' / recode / out_fn

    # Instantiate object
    xlwriter = pd.ExcelWriter(path)

    for ind_var in ind_var_list:

        output = df.groupby(['Year', ind_var]).apply(mean_wt, var, wt=wt).unstack().transpose()
        
        sheet_name = str(ind_var) + '_weighted'
        output.to_excel(xlwriter, sheet_name=sheet_name)

    xlwriter.close()

c:\Users\Lindsey\OneDrive - A3DI\Alive & Thrive\3_Country_II\tabulate_vars.py:38: DtypeWarning: Columns (24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


The file -- LAO_children_combined.csv -- has the following shape: Rows: 32075; Columns: 52


In [13]:
var = 'excl_bf'
ind_var = 'mother_edu_biv'
wt = 'chweight'

df.groupby(['Year', ind_var]).apply(mean_wt, var, wt=wt).unstack().transpose()

Year,2000,2006,2012,2017
mother_edu_biv,,,,
None_Primary,23.7,26.6,39.3,46.1
Secondary_Higher,17.6,26.7,42.8,43.4
